## **Spanish Wine Analysis** 


### **Team members:**
Sindhura Uppalapati, Robert Zhang, Anmol Srivastava, Sohil Jain, Boyue Li

###**Motivation**
According to a recent trend, the Spanish wine market is experiencing a steady decline beginning in 2022. Based on preliminary research, this was due to inefificient supply chain management and lack of understanding of the demand for various categories of wine. Although people's preferences for wine vary greatly when compared to other beverages, we hope to provide valuable insights and recommendations to distributors through this project.

### **Objective:** 
Our objective is to understand the wine description metrics' influence on the item's perceived value and further provide insights on the high performing characteristics by analyzing a dataset that contains information on 7500 different types of wines.

### **Target End Users:**
We aim our study to be helpful to distributors or resellers (those who inherit Spanish wine bottles from wineries and want to sell them) decide to take on new wine products or set their prices. Furthermore, new entrants can use this analysis to get a sense of the market before deciding on the specifications and types of wine with which they want to work.

###**Report Summary**
After research about the key factors wine distributors and resellers are seeking, we analyzed the data keeping the criteria in mind. We found a few trends we believe would be useful for them. Based on these trends and information about the wine supply chain, we've provided recommendations on which wineries are the most profitable, which wines have more scope, and which varieties they should put a hold on. We've also analysed the consumer response for different categories that would be very helpful to them.

### **Raw Data Profile:**
The dataset contains information on 7500 different types of wines on Kaggle that are essentially categorized by the winery from which they were sourced, the product's name, the year in which the grapes used to make the wine were harvested, the type of red wine, and two aspects of the flavor of the wine, body and acidity. The dataset also has two columns to help understand the feedback received. 

#### **Attribute Information-**

**winery**: Name of the Winery the wine is sourced from
wine: Name of the wine

**year**: Year in which the grapes used to make the wine were harvested

**rating**: Average rating given to the wine by the users [from 1-5]

**num_reviews**: Number of users that reviewed the wine

**country**: Country of origin of the wine

**region**: Region of the wine

**price**: Price in euros

**type**: Wine variety

**body**: Body score, defined as the richness and weight of the wine in your mouth [from 1-5]

**acidity**: Acidity score, defined as wine's “pucker” or tartness; it's what makes a wine refreshing and your tongue salivate and want another sip [from 1-5]





### **Data Source Citation :** 
fedesoriano. (April 2022). Spanish Wine Quality Dataset. Retrieved [25 September 2022] from https://www.kaggle.com/datasets/fedesoriano/spanish-wine-quality-dataset


### **1. Data Preprocessing:**
The wine data was gathered by web scraping from several sources, including supermarkets and wine-specific web pages. Some records, therefore, have missing data and outlier values, some records contain duplicates and some columns don't contribute much value to the analysis. We'll work on dropping unnecesary columns, handling missing values, removing duplicate rows and dealing with outliers. Let's go over the step-by-step procedures to clean the data so it is ready for our analysis. 

Before we begin cleaning, given below is an example of random selection for few records:

*Note1- All data processing performed on Google Big Query.*

*Note2- All queries are limited to the top 5 columns to keep the report brief*

In [2]:
#Adding this code to be able to link to bigquery database
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
%%bigquery --project ba778-fall22-team-a8
SELECT * FROM `ba778-fall22-team-a8.project1.spanish-wine`
WHERE rand() <0.01
LIMIT 5

,winery,wine,year,rating,num_reviews,region,price,type,body,acidity
0,Binigrau,Nounat,2020,4.2,404,Mallorca,19.90,NA,NA,NA
1,Binigrau,Nounat,2020,4.2,404,Mallorca,19.90,NA,NA,NA
2,Binigrau,Nounat,2020,4.2,404,Mallorca,19.90,NA,NA,NA
3,Albet i Noya,Marti Reserva Penedes,2015,4.4,49,Penedes,35.15,Red,4,3
4,Clos Pons,Roc Nu,2011,4.2,420,Costers del Segre,23.95,Red,4,3


##### **1.1 Deleting Columns with 0 Standard Deviation**
After analysing the dataset we could observe that the *country* column adds no insight into the data as there it has only one value, i.e., Spain. Below is the query to get the distinct *country* values in the dataset

Hence, dropping *country* column using the following query 

In [ ]:
%%bigquery --project ba778-fall22-team-a8
ALTER TABLE `ba778-fall22-team-a8.project1.spanish-wine`
DROP COLUMN IF EXISTS country

#This query will retrun an error because the column has already been dropped and there's a bug in bigquery that doesn't let us alter table after deleting. 
#Attaching the link about the bug https://stackoverflow.com/questions/68984952/bigquery-drop-table-column-ddl-bug

##### **1.2 Replacing Missing Values**
Next, we want to see if there are any null values in our data so that we can either remove them or replace them with other logical values, if necessary. After previewing the raw data, we noticed there are numerous null values named "NA" for *type*, *body*, and *acidity*. Here is a display of these null values.

In [ ]:
%%bigquery --project ba778-fall22-team-a8
SELECT
  *
FROM
  `ba778-fall22-team-a8.project1.spanish-wine`
WHERE
  type = 'NA'
  OR body = 'NA'
  OR acidity = 'NA'
LIMIT 5

,winery,wine,year,rating,num_reviews,region,price,type,body,acidity
0,Sebio,Salvaxe,2018,4.3,35,Ribeiro,26.75,NA,NA,NA
1,Sebio,Heaven & Hell,2017,4.3,32,Galicia,18.50,NA,NA,NA
2,Sebio,Salvaxe,2017,4.3,31,Ribeiro,28.35,NA,NA,NA
3,Lustau,Vermut Rojo,N.V.,4.2,1058,Jerez-Xeres-Sherry,11.90,NA,NA,NA
4,A Coroa,200 Cestos Godello,2020,4.3,33,Valdeorras,23.70,NA,NA,NA


As we can see from the null values table above, there are a total of 602 records where *type*, *body*, and *acidiy* are unavailable for data analysis. Since our objective includes how these variables may impact perceived *prices*, *number of reviews*, and *ratings*, we decided to handle some of them.

Starting of with *body* and *acidity*, we noticed that this column depends on the type of wine and not the other columns. The following query should help understand this dependence.

In [ ]:
%%bigquery --project ba778-fall22-team-a8
SELECT type, STRING_AGG(DISTINCT body) body, STRING_AGG(DISTINCT acidity) acidity
FROM `ba778-fall22-team-a8.project1.spanish-wine`
GROUP BY type
LIMIT 5

,type,body,acidity
0,NA,NA,NA
1,Red,"4,NA","3,NA"
2,Cava,"2,NA","3,NA"
3,Syrah,"4,NA","3,NA"
4,Mencia,"3,NA","3,NA"


So we create columns *updated_body* and *updated_acidity* to have the new values replacing the "NA" so that they can be used in the future, if necessary. However, we are not updating the original columns to be sure we're aware of the values we're updating and the values that were originally present.

The query to update these columns -

*Note3- We've introduced a table cleaned_data to add all the cleaned data so that the old table can retain our original data information*

In [ ]:
%%bigquery --project ba778-fall22-team-a8
ALTER TABLE `ba778-fall22-team-a8.project1.cleaned_data` ADD COLUMN IF NOT EXISTS updated_body STRING;
UPDATE `ba778-fall22-team-a8.project1.cleaned_data` D1
SET
  updated_body =(
  SELECT MIN(body)
  FROM `ba778-fall22-team-a8.project1.cleaned_data` D2
  WHERE D2.type=D1.type
  GROUP BY D2.type)
WHERE body='NA';

#Similarlu update acidity
ALTER TABLE `ba778-fall22-team-a8.project1.cleaned_data` ADD COLUMN IF NOT EXISTS updated_acidity STRING;
UPDATE `ba778-fall22-team-a8.project1.cleaned_data` D1
SET
  updated_acidity =(
  SELECT MIN(acidity)
  FROM `ba778-fall22-team-a8.project1.cleaned_data` D2
  WHERE D2.type=D1.type
  GROUP BY D2.type)
WHERE acidity='NA';

""


After we replaced these values we still found records where these values were missing but much lesser in number. So we went ahead and deleted those records.

In [ ]:
%%bigquery --project ba778-fall22-team-a8
DELETE 
FROM `ba778-fall22-team-a8.project1.cleaned_data`
WHERE updated_body = 'NA' OR updated_acidity = 'NA';

""


##### **1.3 Deleting records with NULL values that cannot be replaced**
After removing nulls from the previous 2 fields, we believe it is also necessary to remove nulls from the *year* column since it is a key predictor of wine prices. Since we found no correlation of year with another column to replace the NULL values, we're going ahead and dropping the missing values.


In [ ]:
%%bigquery --project ba778-fall22-team-a8
DELETE 
FROM `ba778-fall22-team-a8.project1.cleaned_data`
WHERE year = 'N.V.'

""


Now the cleaned data has a total of 6674 rows of records, which is 826 rows fewer compared to the original 7500 rows of records. This cleaned data is now ready for further analysis and visualization.

In [ ]:
%%bigquery --project ba778-fall22-team-a8
SELECT * 
FROM `ba778-fall22-team-a8.project1.cleaned_data`
LIMIT 5

,winery,wine,year,rating,num_reviews,country,region,price,type,body,acidity,updated_acidity,updated_body
0,Gramona,Cava Celler Batlle,2009,4.4,102,Espana,Cava,59.75,Cava,2,3,None,None
1,Gramona,Cava Enoteca Finca de L'Origen Brut Nature,2002,4.5,28,Espana,Cava,142.00,Cava,2,3,None,None
2,Gramona,Cava Celler Batlle,2006,4.4,201,Espana,Cava,51.95,Cava,2,3,None,None
3,Gramona,Cava Celler Batlle,2010,4.4,126,Espana,Cava,58.30,Cava,2,3,None,None
4,Gramona,Cava Enoteca Finca La Plana Brut Nature,2001,4.6,58,Espana,Cava,164.66,Cava,2,3,None,None


### **2. Data Analysis**

To achieve our objective, we chose aspects in type of the wine, age of the wine, body and acidity to observe affects on customer feedback.

We performed a few basic queries to understand our data better before er get to deeper analysis

#### **2.1. Basic Analysis**

##### **2.1.1. What are the distinct wine options available irrespective of the winery and source**

In [ ]:
%%bigquery --project ba778-fall22-team-a8

SELECT wine, year, price, type, COUNT(*) as Count 
FROM `ba778-fall22-team-a8.project1.cleaned_data` 
GROUP BY wine, year, price, type
LIMIT 5

,wine,year,price,type,Count
0,Cava Celler Batlle,2009,59.75,Cava,1
1,Cava Enoteca Finca de L'Origen Brut Nature,2002,142.00,Cava,1
2,Cava Celler Batlle,2006,51.95,Cava,1
3,Cava Celler Batlle,2010,58.30,Cava,1
4,Cava Enoteca Finca La Plana Brut Nature,2001,164.66,Cava,1


Here, we can gain some understanding with the different and unique combinations of the *wine*, *year* and *price*. There are total 1853 unique rows with the above three combinations.

#####**2.1.2. How many distinct wineries exist in Spain?**

In [ ]:
%%bigquery --project ba778-fall22-team-a8
select distinct(winery) from `ba778-fall22-team-a8.project1.cleaned_data`
LIMIT 5

,winery
0,Gramona
1,Guilera
2,Llopart
3,Mestres
4,Rimarts


We can see with the above query output that we have 411 unique wineries in Spain.

#####**2.1.3. How many distinct wines exist per winery?**

In [ ]:
%%bigquery --project ba778-fall22-team-a8
SELECT
  winery,
  count(DISTINCT(wine)) as number_of_wines
FROM
  `ba778-fall22-team-a8.project1.cleaned_data`
GROUP BY
  winery
ORDER BY number_of_wines DESC
LIMIT 5

,winery,number_of_wines
0,Bodegas Valduero,9
1,Artadi,8
2,Remirez de Ganuza,8
3,Alvaro Palacios,8
4,Marques de Riscal,7


For the above 411 wineries, each winery produces atmost 9 different wines, Bodegas Valduero stands at the top.

#####**2.1.4. What are the number of distinct wines made for each year of grapes harvested?**

In [ ]:
%%bigquery --project ba778-fall22-team-a8 

SELECT COUNT(DISTINCT(wine)) AS number_of_wines, year
FROM `ba778-fall22-team-a8.project1.cleaned_data`
GROUP BY year
ORDER BY number_of_wines DESC
LIMIT 5

,number_of_wines,year
0,217,2016
1,205,2017
2,186,2015
3,157,2018
4,141,2014


From the year 1969 to 2021, the highest production of wines, 217 in number, are with the grapes which were harvested in 2016.

#####**2.1.5. Understanding customer response for each wine**

In [ ]:
%%bigquery --project ba778-fall22-team-a8  
SELECT DISTINCT(wine) AS wine_names, rating, num_reviews
FROM `ba778-fall22-team-a8.project1.cleaned_data`
ORDER BY rating DESC, num_reviews DESC
LIMIT 5

,wine_names,rating,num_reviews
0,Tinto,4.9,58
1,Vina El Pison,4.9,31
2,Unico,4.8,1793
3,Unico,4.8,1705
4,Unico,4.8,1309


The wine "Tinto" and "Vina El Pison" have the highest ratings of all the 1853 wines. 

#### **2.2. Distribution Analysis**
In this section we'll be answering few questions that will help us reach our objective.
#####**2.2.1. Taken year as an important factor, are older wines from the same region more expensive than recently grabbed ones?**
To arrive at the solution, we wrote sub-queries to figure out which region has the highest number of wines to have a higher sample size.

a. *region_with_highest_unique_wines* has the region selling the highest number of unique wines. (It is observed that region '*Rioja*' is the region which has highest count of unique wines)

b. The next subquery was to get the highest selling wine in the region selling the highest number of unique wines. (For the region *Rioja*, the wine with the highest count is '*Reserva*')

c. Next we wrote a main query to compare prices for this wine year over year ("Y-o-Y") for the region selling the highest number of unique wines.

In [ ]:
%%bigquery --project ba778-fall22-team-a8
WITH region_with_highest_unique_wines AS
 (SELECT region,
  FROM `ba778-fall22-team-a8.project1.cleaned_data`
  GROUP BY region
  ORDER BY COUNT(DISTINCT(wine)) DESC
  LIMIT 1)

SELECT wine, year, AVG(price) AVERAGE_PRICE,
  AVG(price)-LAG(AVG(price)) OVER(ORDER BY year DESC) AS DIFFERENCE_IN_PRICE
FROM `ba778-fall22-team-a8.project1.cleaned_data`
WHERE region =(SELECT region FROM region_with_highest_unique_wines)
AND wine = (SELECT wine
            FROM `ba778-fall22-team-a8.project1.cleaned_data`
            WHERE region = (SELECT region FROM region_with_highest_unique_wines)
            GROUP BY wine
            ORDER BY count(wine) DESC
            LIMIT 1)
GROUP BY wine, year
ORDER BY year DESC

,wine,year,AVERAGE_PRICE,DIFFERENCE_IN_PRICE
0,Reserva,2016,19.98,NaN
1,Reserva,2015,17.50,-2.48
2,Reserva,2014,21.50,4.00
3,Reserva,1995,55.00,33.50
4,Reserva,1970,72.00,17.00


As we can see in the result above, the price of the same wine harvested in the same region increases as the age of the harvested wine increases. Thus, taking year as an important factor, we can conclude that the older wines harvested from the same region are more expensive than the recently harvested ones. 

![picture](https://user-images.githubusercontent.com/94079366/196534692-94388caf-e66e-42a1-bc28-4eb2ba5b2e3b.png)

#####**2.2.2. Do wine prices and ratings have a positive or negative correlation?**

In [ ]:
%%bigquery --project ba778-fall22-team-a8 
SELECT wine AS wine_names, rating,
       AVG(price) AS average_price,
       AVG(price)-LAG(AVG(price)) OVER(ORDER BY rating DESC) AS DIFFERENCE_IN_PRICE
FROM `ba778-fall22-team-a8.project1.cleaned_data`
GROUP BY wine_names,rating
ORDER BY rating DESC
LIMIT 5;


,wine_names,rating,average_price,DIFFERENCE_IN_PRICE
0,Tinto,4.9,995.00000,681.50000
1,Vina El Pison,4.9,313.50000,NaN
2,Pingus,4.8,1215.00000,216.00000
3,El Nido,4.8,195.62738,-934.01512
4,Cuesta de Las Liebres,4.8,166.18000,61.03000


As we can see in the result above, the pattern of difference in prices is highly asymmetrical. This is because the price bracket for different wines are diffrent, which does not provide an accurate difference in prices for wines with different ratings. Hence, the data is highly asymmetrical and we cannot give a recommendation to a distributor purely based on price. 


![picture](https://user-images.githubusercontent.com/94079366/196534779-b5c92978-890d-4dfa-81a4-a26bee6779a9.png)

#####**2.2.3. Does the wine with the highest number of ranks in body and acidity actually have highest/nearly good ratings?**

To arrive at the solution, we first write a subquery to filter the wines in descending order, with highest number of body and acidity rankings. The wine with highest ranks among all is '*Fos Baranda*'

Next, we compare the ratings of the same wines, if the highest ranking also has the highest rating or not

In [ ]:
%%bigquery --project ba778-fall22-team-a8  
SELECT DISTINCT(wine) AS wine, rating,
FROM `ba778-fall22-team-a8.project1.cleaned_data`
WHERE body=(SELECT MAX(body)
            FROM `ba778-fall22-team-a8.project1.cleaned_data`)
AND acidity=(SELECT MAX(acidity)
                         FROM `ba778-fall22-team-a8.project1.cleaned_data`)
GROUP BY wine, rating
HAVING wine=(SELECT DISTINCT(wine) AS wine,
             FROM `ba778-fall22-team-a8.project1.cleaned_data`
             WHERE body=(SELECT MAX(body)
                          FROM `ba778-fall22-team-a8.project1.cleaned_data` )
             AND acidity=(SELECT MAX(acidity)
                          FROM `ba778-fall22-team-a8.project1.cleaned_data`)
             GROUP BY wine
             LIMIT 1)
ORDER BY rating DESC

,wine,rating
0,Fos Baranda,4.3


As we can observe in the result above, we see the wine with highest ranking in body and acidity is *Fos Baranda*, but when compared with ratings, it doesn't have the highest rating. Thus, we can conclude the wine with highest number of ranks in body and acidity actually does not have the  highest or nearly good ratings and the distributor cannot make substansial decisions based off body and acidity alone.

#####**2.2.4. Does the winery with the highest production of wines come under the region with which produces the maximum number of wines?**

To arrive at the solution, we first wrote a subquery that returns the name of the winery with highest number of wines (The winery with the highest number of wines is '*Contino*'). 
As we noted in the earlier analysis, the region with highest count of wines is *Rioja*. We checked the region of the winery with the highest selling wines.

In [ ]:
%%bigquery --project ba778-fall22-team-a8 
-- check with winery “Contino” in the region with highest selling wines
SELECT MAX(region)
FROM `ba778-fall22-team-a8.project1.cleaned_data`
WHERE winery = (SELECT winery
                FROM `ba778-fall22-team-a8.project1.cleaned_data`
                GROUP BY winery
                ORDER BY COUNT(wine) DESC
                LIMIT 1)

,f0_
0,Rioja


As we can note above, the winery with highest number of count *'Contino'* is present in the region with highest count of wines, *'Rioja'*. Thus, we can suggest the distributors to build supply chain channels in these regions because they're producing a lot of wines and the same channel can be used for all of them.

##### **2.2.5. Which region produces the cheapest wines with the highest ratings?**

In [ ]:
%%bigquery --project ba778-fall22-team-a8 

SELECT region, AVG(price) AS averageprice
FROM `ba778-fall22-team-a8.project1.cleaned_data`
WHERE rating = (SELECT MAX(rating)
                FROM `ba778-fall22-team-a8.project1.cleaned_data`)
GROUP BY region
ORDER BY averageprice ASC
LIMIT 1

,region,averageprice
0,Vino de Espana,313.5


As we can note above, the region which has the cheapest wine with highest rating is '*Vino de Espana*'.

#####**2.2.6 Which winery has the highest consumers and rating?**
Here we are making an assumption that higher the number of reviews more the number of people consuming that wine. So in this step we are trying to find the winery that has the highest consumers and a good rating.

In [ ]:
%%bigquery --project ba778-fall22-team-a8 
SELECT DISTINCT winery,
  (SUM(num_reviews) OVER(PARTITION BY winery)) AS reviews,
  ROUND(AVG(price) OVER(PARTITION BY winery),2) AS avg_price,
  ROUND(AVG(rating) OVER(PARTITION BY winery),2) AS avg_rating
FROM `ba778-fall22-team-a8.project1.cleaned_data`
GROUP BY winery, num_reviews, price, rating
ORDER BY reviews DESC
LIMIT 5

,winery,reviews,avg_price,avg_rating
0,Vega Sicilia,117663,601.18,4.67
1,La Rioja Alta,83634,101.24,4.36
2,Pintia,44832,91.07,4.37
3,Alion,41996,121.43,4.46
4,Dominio de Pingus,39851,796.43,4.55


We can notice that *'Vega Sicilia'* has a very high engagement rate which is proportional to the number of consumers and a much higher average rating for it's wines. Doing business with such high performing wineries will be very useful for the distributors.

#####**2.2.7. Which wineries are not receiving encouraging feedback from consumers?**

In [ ]:
%%bigquery --project ba778-fall22-team-a8 
SELECT DISTINCT winery
FROM (
SELECT DISTINCT winery,
  (SUM(num_reviews) OVER(PARTITION BY winery)) AS reviews,
  ROUND(AVG(price) OVER(PARTITION BY winery),2) AS avg_price,
  ROUND(AVG(rating) OVER(PARTITION BY winery),2) AS avg_rating
FROM `ba778-fall22-team-a8.project1.cleaned_data`
GROUP BY winery, num_reviews, price, rating
ORDER BY reviews asc)
WHERE avg_rating<4.5
LIMIT 5

,winery
0,Senorio de San Vicente
1,Merum Priorati
2,Mas Perinet
3,Comenge
4,Belondrade


Based on the results above, the winery 'Senorio de San Vicente' has not received the most encouraging feedback from consumers.

The following visualization helps understand 2.2.6 and 2.2.7.

![picture](https://user-images.githubusercontent.com/94079366/196534647-c3620191-e205-4461-9812-9e638be0c46e.png)

#####**2.2.8 Which types of wine are people trying the most and what is their review?**

In [ ]:
%%bigquery --project ba778-fall22-team-a8 
SELECT * FROM
(SELECT type, SUM(num_reviews) AS total_reviews, AVG(rating) AS avg_rating,
RANK() OVER(ORDER BY SUM(num_reviews) DESC) AS rank_by_number_of_consumers
FROM `ba778-fall22-team-a8.project1.cleaned_data`
GROUP BY type
ORDER BY total_reviews DESC)
WHERE TYPE like '%Red%'

,type,total_reviews,avg_rating,rank_by_number_of_consumers
0,Rioja Red,1101705,4.232160,1
1,Ribera Del Duero Red,808717,4.296446,2
2,Red,357154,4.242758,3
3,Priorat Red,225805,4.270623,4
4,Toro Red,165567,4.264189,5
5,Montsant Red,1803,4.352941,18


We can notice in our above query that a lot of people keep trying different varieties of Red Wine but get disappointed by it and hence the low ratings. Distributors should look into new avenues and wineries that produce better red wines that consumers prefer. The number of people trying out red wine signifies the amount of market it has.

![picture](https://user-images.githubusercontent.com/94079366/196534752-ea531cc5-66f9-47c5-b251-44e1d8bbee54.png)

##### **2.2.9. Which regions have the highest number of high performing wineries?**

In [ ]:
%%bigquery --project ba778-fall22-team-a8
SELECT region, COUNT(winery) as no_of_high_performing_wineries
FROM ( SELECT DISTINCT winery,region,
       (SUM(num_reviews) OVER(PARTITION BY winery)) AS reviews,
       ROUND(AVG(price) OVER(PARTITION BY winery),2) AS avg_price,
       ROUND(AVG(rating) OVER(PARTITION BY winery),2) AS avg_rating
       FROM `ba778-fall22-team-a8.project1.cleaned_data`
       GROUP BY winery, num_reviews, price, rating, region )
WHERE avg_rating>4.5
GROUP BY region
ORDER BY no_of_high_performing_wineries DESC
LIMIT 5

,region,no_of_high_performing_wineries
0,Ribera del Duero,4
1,Toro,2
2,Priorato,2
3,Rioja,2
4,Vino de Espana,1


Establishing a trade channel in regions like *'Ribera del Duero'* would be a smart move because there are a lot of wineries in that region that are performing well.

![picture](https://user-images.githubusercontent.com/94079366/196534726-fd546237-0ffd-49d7-a3f5-a3921bcd8b31.png)

### **Conclusions**
In our analysis on wines, wineries, price, and customer response, we report the following findings that are aimed towards helping distributors who are invlolved in spanish wine distribution make customer friendly choices(Supporting data is available in the queries)-

1. Distributors should pick wineries that are doing well because it costs the them money to create and maintain a supply chain channel that allows them to source wines from each winery. It's a smarter choice to do business with wineries that produce consistently highly rated wines and those that get the most interest. We could notice that *'Vega Sicilia'* is one such well performing winery according to our datset.

2. Similar to the previous point, distributors should put a hold on working with low performing wineries. The investment into maintaining a supply chain channel to these wineries does not yield as much profit. Few such wineries we found in our analysis are *Francisco Barona*, *Pago de Vallegarcia*, *Bodegas Naluar & Acediano*, *Hacienda Grimon*, *Pago de Los Capellanes*.

3. We observed that all types of red wine have a large number of customers, but they are mostly not satisfied, as evidenced by the low ratings. More research and procurement has to be done in this area as red wine displays a huge potential according to our dataset.

4. Establishing a trade channel in multiple wineries of the same region is usually more profitable logistically. Instead of maintaining trade channels in regions with very few or no well-performing wineries, we recommend working in regions such as *Ribera del Duero*, which has the highest number of distinct wines, and *Rioja*, which has the highest number of distinct wines. This way they can either cover wines that are mostly loved by consumers or make sure they've a wide variety of options with optimal number of trade channels.


### **Dashboard**

We've created a Tableau dashboard that shows the trends in our data that we based our conclusions on. A few sheets that are a part of this dashboard have already been included in the document already.

Link to Tableau Dashboard- https://public.tableau.com/app/profile/robert.zhang7948/viz/shared/XKD9H98SN

Following are the screenshots of the same

![picture](https://user-images.githubusercontent.com/23219748/196556991-c46156d4-6f1f-4dc6-8e1e-ee397b5a95de.png)

![picture](https://user-images.githubusercontent.com/23219748/196557283-61f04c2e-56e4-47cf-abf9-ff103e6d83d6.png)

### **References**

To better understand the dataset-
https://www.kaggle.com/datasets/fedesoriano/spanish-wine-quality-dataset

Spanish wine trends- https://tastewise.io/foodtrends/spanish%20wine

Research on existing wine distributors to better understand their goals-
1. https://www.spanishwinelover.com/find-137-us-importers-who-champion-spanish-wines#
2. https://spainwinecollection.com/

Supplementary information on supply chain for wine to aid our analysis-
1. https://rfxcel.com/wine-supply-chain-trends-2022/
2. https://www.datexcorp.com/supply-chain-management-basics-wine-and-spirits-logistics/#:~:text=The%20three%20main%20tiers%20of,stores%2C%20grocers%2C%20restaurants).
3. https://www.reuters.com/business/retail-consumer/supply-chain-crisis-threatens-christmas-spanish-wines-2021-11-15/

**Note- Assumptions based of these sources were mentioned as and when used in the analysis.**




**Following is an additional section containing Data Visualisations that we used for our preliminary analysis but are very basic. Hence adding them as supplementary information.**

### **3. Data Visualization**
In this section we use visualisation tools to better understand our data visually.

*Note4- Tableau was used for all data visualisations.*

##### **3.1. Which winery in Spain produced the most diverse portfolio of wines, and below is a brief ranking of top 10 wineries.**

In [ ]:
%%bigquery --project ba778-fall22-team-a8  
SELECT winery, COUNT(DISTINCT wine) AS wine_selection,
FROM `ba778-fall22-team-a8.project1.cleaned_data`
GROUP BY winery
ORDER BY wine_selection DESC
LIMIT 5;

,winery,wine_selection
0,Bodegas Valduero,9
1,Artadi,8
2,Remirez de Ganuza,8
3,Alvaro Palacios,8
4,Marques de Riscal,7


![picture](https://user-images.githubusercontent.com/94079366/193908427-ae29597e-b8b7-45ad-aa67-1f09fc6f54db.png)

##### **3.2 Which region in Spain produces the most expensive wine and rank regions based on descending prices.**
Based on this ranking, we produced the following bubble chart to visualize which region has the highest total revenue. In addition, the top 3 regions Ribera del Duero, Rioja, and Priorato are all located in the northern part of Spain renowned for their wine reputation. 

In [ ]:
%%bigquery --project ba778-fall22-team-a8  
SELECT region, ROUND(SUM(price),2) AS total_price
FROM `ba778-fall22-team-a8.project1.cleaned_data`
GROUP BY region
ORDER BY total_revenue desc
LIMIT 5

,region,total_revenue
0,Ribera del Duero,154275.60
1,Rioja,107602.24
2,Priorato,70939.30
3,Toro,22808.56
4,Vino de Espana,19262.63


![picture](https://user-images.githubusercontent.com/94079366/193915646-cfb85075-277b-4bf2-937a-c073d3b5aa42.png)

**3.3 Relationship between different regions and ratings**
Since we ranked regions by revenue, we wanted to see if the top revenue-generating regions also have higher wine ratings. If so, it means there is a positive correlation between ratings and revenues. As shown below, the top 3 revenue-generating wineries identified in the previous query also generate the highest median ratings, suggesting that ratings and wine revenues are positively correlated. 

In [ ]:
%%bigquery  --project ba778-fall22-team-a8  
SELECT DISTINCT winery, region, ROUND(PERCENTILE_DISC(rating, 0.5) OVER(PARTITION BY winery), 2) AS median_rating
FROM `ba778-fall22-team-a8.project1.cleaned_data`
GROUP BY winery, region, rating
ORDER BY median_rating DESC
LIMIT 5

,winery,region,median_rating
0,Vina Real,Rioja,4.6
1,Micro Bio (MicroBio),Vino de Espana,4.6
2,Vega Sicilia,Ribera del Duero,4.6
3,Acustic Celler,Priorato,4.6
4,Teso La Monja,Toro,4.6


![picture](https://user-images.githubusercontent.com/94079366/194125058-9a6035e3-6de4-4631-9990-487e63fb59b2.png)

#####**3.4. Relationship between age of wine and price**

We also investigated if there is a positive relationship between year and price. Our histogram below supports the general belief that the older the wine, the higher the prices, keeping the country of origin constant. As illustrated in the descending and ascending tables, the median wine price in 1965 is the highest at €1,515 per bottle, while the median wine price in 2021 is only €13 per bottle. The reason for using median wine price as a metric is to remove outlier's impact.

![print](https://user-images.githubusercontent.com/94079366/193955202-246aa10a-bc23-4a34-9ca8-7bd9058ba371.png)